In [1]:
import numpy as np                   # Basic Array and Numaric Operation
import os                            # use to access the files 
import tarfile                       # use to extract dataset from zip files
import sys
import zipfile                       # to extract zip file


import torch                         # Provides basic tensor operation and nn operation
import torchvision                   # Provides facilities to access image dataset


import my_utils.loadDataset as dl           # create dataloader for selected dataset
import my_utils.loadModel as lm             # facilitate loading and manipulating models
import my_utils.trainModel as tm            # Facilitate training of the model
import my_utils.initialize_pruning as ip

### Prepare dataloader and set image properties. Check for cuda device and load model

In [4]:
dl.setFolderLocation(datasets       ='/home/pragnesh/Dataset/',
                     selectedDataset='IntelIC/',
                     train          ='train',
                     test           ='test')
dl.setImageSize(224)
dl.setBatchSize = 2
dataLoader = dl.dataLoader()

#load_path = "/home/pragnesh/Model/Model/VGG_IntelIC_v2"
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
newModel = torch.load(load_path, map_location=torch.device('cpu'))

# newmodel = lm.load_model(model_name='vgg16',number_of_class=6,pretrainval=False,
#                         freeze_feature=False,device_l=device1)

print(newModel)

### Here we have 3 set of parameter.One for data loader, second for model and third are hyperparameter
### Train the model

In [ ]:
logFile = '/home/pragnesh/Dataset/Intel_Image_Classifacation_v2/Logs/ConvModelv2.log'
#tm.device = torch.device('cpu')
tm.fit_one_cycle(dataloaders=dataLoader,trainDir=dl.trainDir,testDir=dl.testDir,
                 ModelName='vgg16',model=newmodel,device_l=device1,
                 epochs=1, max_lr=0.01, weight_decay=0, L1=0, grad_clip=.1, logFile=logFile)
SavePath = '/home/pragnesh/Model/vgg16-v2'
torch.save(newModel, SavePath)

### Pruning Initialization

In [6]:
blockList   = ip.createBlockList(newModel)#ip.getBlockList('vgg16')
featureList = ip.createFeatureList(newModel)
module = ip.getPruneModule(newModel)
print(f"blockList   = {blockList}\n"
      f"featureList = {featureList}\n"
     )
for i in range(len(module)):
    print(module[i])

blockList   = [2, 2, 3, 3, 3]
featureList = [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28]

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
"""Select the amount of feature we want to prune in each Layer"""
# epochs = 10
# max_lr = 1e-3
# grad_clip = .2 
# weight_decay = 1e-5 
# L1=1e-5
# itteration = 6
# for  ittr in range(itteration):
#     for i in range(len(Module)):
#         prune.ln_structured(Module[i], name="weight", amount=pr[i]/itteration, n=1, dim=0)
#     numberOfZero = 0
#     numberOfElements = 0
#     totalNumberOfZero=0
#     totalNumberOfElements=0
#     for i, j in zip(range(len(prunelist)),prunelist):
#         numberOfZero = torch.sum(Module[i].weight == 0)
#         totalNumberOfZero += numberOfZero
#         numberOfElements = Module[i].weight.nelement()
#         totalNumberOfElements += numberOfElements
#         frac = 100. * float(torch.sum(Module[i].weight == 0))/float(Module[i].weight.nelement())
#         with open(outfile,'a') as f:
#             f.write((f"{j} Sparsity in {Module[i]} is \t{frac}"))
        
#     with open(outfile,'a') as f:
#         f.write(("Global Sparsity: {:.2f}%".format(100*float(numberOfZero)/float(numberOfElements))))

#     print(f"******************** {ittr}**********************")
#     #fit_one_cycle(ModelName,epochs, max_lr, model,weight_decay=0, 
#     #             L1=0,grad_clip=None, opt_func=torch.optim.SGD):
#     historyPrun = fit_one_cycle(MODEL_NAME,epochs, max_lr, newModel,  
#                               grad_clip=grad_clip, 
#                               weight_decay=weight_decay, L1=L1,
#                               opt_func=opt_func
#                               )

In [ ]:
# prune_count= [3,3, 6,6, 13,13,13, 26,26,26, 26,26,26]
# j=0
# for i in range(len(vgg16_features)):
#     if vgg16_features[i]=='M':
#         continue
#     else:
#         vgg16_features[i] = vgg16_features[i]-prune_count[j]
#         j+=1
vgg16Model = lm.create_vgg_from_feature_list(vgg16_features)    